<a href="https://colab.research.google.com/github/margaretmz/E2E-tfKeras-TFLite-Android/blob/master/ml/MNIST_tfKeras_TFLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End to End: tf.Keras to TFLite to Android with MNIST

By Margaret Maynard-Reid - created on 1/29/2019 and updated on 10/4/2019

This tutorial demonstrates the end to end process of how to:

*   train a simple CNN model to classify MNIST digit with [tf.keras],(https://www.tensorflow.org/guide/keras)
*   save a Keras model,
*   convert the Keras model to tflite with the TOCO converter,
*   download the tflite model for deploying to a DigitRecognizer Android app.



>[End to End: tf.Keras to TFLite to Android with MNIST](#scrollTo=oY3YSP69c8Hr)

>>[Data](#scrollTo=E9RImw1qa-DH)

>>>[Get MNIST dataset](#scrollTo=P4vs-cagt0XD)

>>>[Preprocess data](#scrollTo=0P1C_0ZXux7N)

>>[Model training](#scrollTo=4mlybelBbT0r)

>>>[Define the model architecture](#scrollTo=JEkp9ZrNwCl0)

>>>[Inspect the model architecture](#scrollTo=RVkPFj5D_qRy)

>>>[Train the model](#scrollTo=o94eQubW6EMb)

>>[Model conversion](#scrollTo=ueEWfvF1bSZX)

>>>[Save a Keras model](#scrollTo=1x0Cxiyb25KA)

>>>[Convert Keras model to TensorFlow Lite](#scrollTo=yi9B0yFR5J1b)

>>>[Download the tflite model for Android](#scrollTo=2lrX0Bj97jor)



In [0]:
# Inport TensorFlow and tf.keras
import tensorflow as tf
keras = tf.keras

# Import helper libraries
import numpy as np
import matplotlib.pyplot as plt

# Print TensorFlow version
tf.__version__

## Data
We will use the MNIST dataset for training the image classifier.

### Get MNIST dataset

In [0]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

### Preprocess data
Next we will reshape the input image from 28x28 to 28x28x1, normalize it and one-hot encode the labels.

In [0]:
num_classes = 10
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize the input data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Reshape input data from (28, 28) to (28, 28, 1)
w, h = 28, 28
x_train = x_train.reshape(x_train.shape[0], w, h, 1)
x_test = x_test.reshape(x_test.shape[0], w, h, 1)

# One-hot encode the labels
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Take a look at the dataset shape after conversion with keras.utils.to_categorical
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

## Model training

### Define the model architecture

There are 3 ways to define a model in tf.Keras:
1. Sequential model API
2. Functional API
3. Model subclassing

We will create a simple Convolutional Neural Network with tf.Keras Sequential model API.

In [0]:
def create_model():
  
  # Define the model architecture
  model = keras.models.Sequential([
    # Must define the input shape in the first layer of the neural network
    keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(28,28,1)),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Dropout(0.3),

    keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Dropout(0.3),

    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  # Compile the model
  model.compile(loss=keras.losses.categorical_crossentropy,
         optimizer=keras.optimizers.Adam(),
         metrics=['accuracy'])
      
  return model

### Inspect the model architecture
Let's take a look at the model architecture

In [0]:
model = create_model()

# Take a look at the model summary
model.summary()

### Train the model

In [0]:
%%time 
model.fit(x_train,
         y_train,
         batch_size=64,
         epochs=3,
         validation_data=(x_test, y_test))

## Model conversion

### Save a Keras model
We will use the save_model() method to save a Keras model in .h5 format

In [0]:
# Save tf.keras model in HDF5 format
keras_model = "mnist_keras_model.h5"
keras.models.save_model(model, keras_model)

 ### Convert Keras model to TensorFlow Lite
 
 Now let's convert the Keras model to TensorFlow Lite format. 
 
 It is recommended that you do the model conversion through Python code; however, in case you don't have access to your model code, you can also convert the model through command line. 

In [0]:
from datetime import date

converter = tf.lite.TFLiteConverter.from_keras_model_file(keras_model) # TF 2.0
tflite_model = converter.convert()
tflite_model_file_name = "mnist_" + tf.__version__ + "_" + str(date.today()) + ".tflite"
open(tflite_model_file_name, "wb").write(tflite_model)

### Download the tflite model for Android
Let's verify the tflite model has been generated and download it to use in your mobile app such as Android.

In [0]:
# Take a look at the files in the directory
# Or you can see them under Colab Files tab
import os
os.listdir(".")

In [0]:
# Download the .tflite model file for deployment
from google.colab import files
files.download(tflite_model_file_name) 